<h1 style="font-size:3rem; color: darkgoldenrod;">Data Cleaning and Preparation:</h1>

## Part 2: Data Transformation

So far in this week we’ve been concerned with rearranging data. Filtering, cleaning, and other transformations are another class of important operations.

# Table of Contents

- 2.1  **[Filtering Out Missing Data](#Removing_Duplicates)**
   
- 2.2  **[Transforming Data Using a Function or Mapping](#Transforming_Data)**

- 2.3  **[Replacing Values](#Replacing_Values)**
   
- 2.4  **[Renaming Axis Indexes](#Renaming_Axis_Indexes)**

- 2.5  **[Discretization and Binning](#Discretization_and_Binning)**

- 2.6  **[Detecting and Filtering Outliers](#Detecting_and_Filtering_Outliers)**
   
- 2.7  **[Permutation and Random Sampling](#Permutation_and_Random_Sampling)**

- 2.8  **[Computing Indicator/Dummy Variables](#Dummy_Variables)**

In [1]:
import pandas as pd
import numpy as np

<a id="Removing_Duplicates"></a>
## 2.1 Filtering Out Missing Data

Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [2]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                   'k2': [1, 1, 2, 3, 3, 4, 4]})

In [3]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method `duplicated` returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [4]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, `drop_duplicates` returns a DataFrame where the `duplicated` array is False:

In [5]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [6]:
data['v1'] = range(7)

In [7]:
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [8]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by default keep the first observed value combination. Passing `keep='last'` will return the last one:

In [9]:
data.drop_duplicates(['k1', 'k2'], keep='last') 

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


<a id="Transforming_Data"></a>
## 2.2 Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:

In [10]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                             'Pastrami', 'corned beef', 'Bacon',
                             'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [11]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food came from. Let’s write down a mapping of each distinct meat type to the kind of animal:

In [27]:
meat_to_animal = {
      'bacon': 'pig',
      'pulled pork': 'pig',
      'pastrami': 'cow',
      'corned beef': 'cow',
      'honey ham': 'pig',
      'nova lox': 'salmon'
}    
meat_to_animal

{'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'}

The `map` method on a Series accepts a function or dict-like object containing a mapping, but here we have a small problem in that some of the meats are capitalized and others are not. Thus, we need to convert each value to lowercase using the `str.lower` Series method:

In [28]:
lowercased = data['food'].str.lower()

In [29]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [30]:
data['animal'] = lowercased.map(meat_to_animal)

In [31]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work:

In [32]:
data['food'].map(lambda x: meat_to_animal[x.lower()]) 

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using `map`is a convenient way to perform element-wise transformations and other data cleaning–related operations.

<a id="Replacing_Values"></a>
## 2.3 Replacing Values

Filling in missing data with the `fillna` method is a special case of more general value replacement. As you’ve already seen, `map` can be used to modify a subset of values in an object but `replace` provides a simpler and more flexible way to do so. Let’s consider this Series:

In [33]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [34]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The *-999* values might be sentinel values for missing data. To replace these with NA values that pandas understands, we can use `replace`, producing a new Series (unless you pass `inplace=True`):

In [35]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the
substitute value:

In [36]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:

In [38]:
# data
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dict:

In [39]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

<a id="Renaming_Axis_Indexes"></a>
## 2.4 Renaming Axis Indexes

Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in-place without creating a new data structure. Here’s a simple example:

In [40]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])


In [41]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Like a Series, the axis indexes have a `map` method:

In [43]:
transform = lambda x: x[:4].upper() 

In [44]:
data.index.map(transform) 

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

You can assign to index, modifying the DataFrame in-place:

In [45]:
data.index = data.index.map(transform)

In [46]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11



If you want to create a transformed version of a dataset without modifying the original, a useful method is `rename`:

In [47]:
data.rename(index=str.title, columns=str.upper) 

#The str.title() function is used to convert strings in the Series/Index to titlecase.

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [48]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Notably, `rename` can be used in conjunction with a dict-like object providing new values for a subset of the axis labels:

In [49]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


`rename` saves you from the chore of copying the DataFrame manually and assigning to its index and columns attributes. Should you wish to modify a dataset in-place, pass `inplace=True`:

In [50]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [51]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


<a id="Discretization_and_Binning"></a>
## 2.5 Discretization and Binning

Continuous data is often discretized or otherwise separated into “bins” for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [52]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
ages

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [53]:
bins = [18, 25, 35, 60, 100]

In [54]:
cats = pd.cut(ages, bins)

In [55]:
cats 

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special `Categorical` object. The output you see describes the bins computed by `pandas.cut`. You can treat it like an array of strings indicating the bin name; internally it contains a `categories` array specifying the distinct category names along with a labeling for the `ages`data in the codes attribute:

In [56]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [57]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [58]:
pd.value_counts(cats)

/var/folders/t6/jzgwrdks2vq53w20jlljrk840000gn/T/ipykernel_37012/1485279302.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(cats)


(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

Note that `pd.value_counts(cats)` are the bin counts for the result of `pandas.cut`.

Consistent with mathematical notation for intervals, **a parenthesis means that the side is open, while the square bracket means it is closed (inclusive)**. You can change which side is closed by passing `right=False`:

In [59]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

You can also pass your own bin names by passing a list or array to the `labels` option:

In [60]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [61]:
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to `cut` instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data. Consider the case of some uniformly distributed data chopped into fourths:

In [63]:
data = np.random.rand(20)

In [64]:
pd.cut(data, 4, precision=2) # The precision=2 option limits the decimal precision to two digits.

[(0.48, 0.71], (0.48, 0.71], (0.71, 0.95], (0.25, 0.48], (0.71, 0.95], ..., (0.48, 0.71], (0.012, 0.25], (0.012, 0.25], (0.25, 0.48], (0.71, 0.95]]
Length: 20
Categories (4, interval[float64, right]): [(0.012, 0.25] < (0.25, 0.48] < (0.48, 0.71] < (0.71, 0.95]]

A closely related function, `qcut`, bins the data based on sample quantiles. Depending on the distribution of the data, using `cut` will not usually result in each bin having the same number of data points. Since `qcut` uses sample quantiles instead, by definition you will obtain roughly equal-size bins:

In [65]:
data = np.random.randn(1000) # Normally distributed

In [66]:
cats = pd.qcut(data, 4) # Cut into quartiles

In [67]:
cats

[(-2.972, -0.644], (0.682, 3.269], (-2.972, -0.644], (0.682, 3.269], (-0.0389, 0.682], ..., (-2.972, -0.644], (0.682, 3.269], (0.682, 3.269], (-2.972, -0.644], (-2.972, -0.644]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.972, -0.644] < (-0.644, -0.0389] < (-0.0389, 0.682] < (0.682, 3.269]]

In [68]:
pd.value_counts(cats)

/var/folders/t6/jzgwrdks2vq53w20jlljrk840000gn/T/ipykernel_37012/1485279302.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(cats)


(-2.972, -0.644]     250
(-0.644, -0.0389]    250
(-0.0389, 0.682]     250
(0.682, 3.269]       250
Name: count, dtype: int64

Similar to `cut` you can pass your own quantiles (numbers between 0 and 1, inclusive):

We’ll return to cut and qcut later in the lecture during our discussion of aggregation and group operations, as these discretization functions are especially useful for quantile and group analysis.

<a id="Detecting_and_Filtering_Outliers"></a>
## 2.6 Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [69]:
data = pd.DataFrame(np.random.randn(1000, 4))

In [70]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.026036,0.081351,0.027962,0.032454
std,1.022113,1.018162,1.060938,0.996455
min,-3.788971,-3.721483,-3.289044,-3.510632
25%,-0.674334,-0.549232,-0.727367,-0.645196
50%,0.004084,0.107705,0.067662,0.041811
75%,0.681864,0.695644,0.733459,0.719405
max,2.956742,3.969886,3.614664,2.582846


Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:

In [71]:
col = data[2]

In [72]:
col

0      1.334387
1     -0.203834
2      0.277208
3      2.245935
4     -0.339900
         ...   
995   -0.189739
996   -0.021289
997    2.452666
998    0.953101
999    1.696563
Name: 2, Length: 1000, dtype: float64

In [73]:
col[np.abs(col) > 3]

58    -3.289044
140    3.014686
343    3.256859
419    3.614664
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the `any` method on a boolean DataFrame:

In [77]:
data[(np.abs(data) > 3)].any(axis=1)

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3:

In [78]:
data[np.abs(data) > 3] = np.sign(data) * 3 

In [79]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.024312,0.079774,0.027365,0.033415
std,1.016621,1.004136,1.057372,0.993290
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.674334,-0.549232,-0.727367,-0.645196
50%,0.004084,0.107705,0.067662,0.041811
75%,0.681864,0.695644,0.733459,0.719405
max,2.956742,3.000000,3.000000,2.582846


The statement `np.sign(data)` produces 1 and –1 values based on whether the values in `data` are positive or negative:

In [80]:
np.sign(data).head()  

,0,1,2,3
0,1.0,-1.0,1.0,-1.0
1,-1.0,1.0,-1.0,1.0
2,1.0,-1.0,1.0,-1.0
3,1.0,-1.0,1.0,-1.0
4,1.0,-1.0,-1.0,-1.0


<a id="Permutation_and_Random_Sampling"></a>
## 2.7 Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the `numpy.random.permutation` function. Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [81]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))

In [82]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [88]:
df = pd.DataFrame(np.arange(32).reshape((4, 8))) 

In [89]:
df


,0,1,2,3,4,5,6,7
0,0,1,2,3,4,5,6,7
1,8,9,10,11,12,13,14,15
2,16,17,18,19,20,21,22,23
3,24,25,26,27,28,29,30,31


In [90]:
sampler = np.random.permutation(5) 

#The permutation() method returns a re-arranged array (and leaves the original array un-changed).

In [91]:
sampler

array([1, 3, 0, 4, 2])

That array can then be used in `iloc`-based indexing or the equivalent `take` function:

In [92]:
df.take(sampler)

IndexError: indices are out-of-bounds

To select a random subset without replacement, you can use the `sample` method on Series and DataFrame:

In [93]:
df.sample(n=3)

,0,1,2,3,4,5,6,7
0,0,1,2,3,4,5,6,7
2,16,17,18,19,20,21,22,23
1,8,9,10,11,12,13,14,15


To generate a sample with replacement (to allow repeat choices), pass `replace=True` to `sample`:

In [94]:
choices = pd.Series([5, 7, -1, 6, 4])

In [95]:
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

In [96]:
draws = choices.sample(n=10, replace=True)

In [97]:
draws

2   -1
1    7
3    6
1    7
3    6
1    7
0    5
2   -1
2   -1
2   -1
dtype: int64

<a id="Dummy_Variables"></a>
## 2.8 Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a “dummy” or “indicator” matrix. 

If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s. pandas has a `get_dummies` function for doing this. Let’s return to an earlier example DataFrame:

In [98]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 
                  'data1': range(6)})

In [99]:
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [100]:
pd.get_dummies(df['key'])

,a,b,c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


In [101]:
# create dataset
df = pd.DataFrame({'Temperature': ['Hot', 'Cold', 'Warm', 'Cold'],
                   })

In [102]:
# display dataset
df

,Temperature
0,Hot
1,Cold
2,Warm
3,Cold


In [103]:
# create dummy variables
pd.get_dummies(df)

,Temperature_Cold,Temperature_Hot,Temperature_Warm
0,False,True,False
1,True,False,False
2,False,False,True
3,True,False,False


In [104]:
# create dataset
s = pd.Series(list('abca'))

In [105]:
# display dataset
s

0    a
1    b
2    c
3    a
dtype: object

In [106]:
# create dummy variables
pd.get_dummies(s)

,a,b,c
0,True,False,False
1,False,True,False
2,False,False,True
3,True,False,False


In [107]:
# create dataset
df = pd.DataFrame({'A': ['hello', 'vignan', 'geeks'],
                   'B': ['vignan', 'hello', 'hello'],
                   'C': [1, 2, 3]})
 

In [108]:
# display dataset
print(df)

        A       B  C
0   hello  vignan  1
1  vignan   hello  2
2   geeks   hello  3


In [109]:
# create dummy variables
pd.get_dummies(df)

,C,A_geeks,A_hello,A_vignan,B_hello,B_vignan
0,1,False,True,False,False,True
1,2,False,False,True,True,False
2,3,True,False,False,True,False


A useful recipe for statistical applications is to combine `get_dummies` with a discretization function like `cut`:

In [110]:
np.random.seed(12345)

In [111]:
values = np.random.rand(10)

In [112]:
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [113]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [114]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False
